# Session 2.2: BakeryAI - Advanced Tools & Structured Output


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LY5vAv33AHRY-FBJaTDVjGHMlmlO9bZa?usp=sharing)


## 🎯 Today's Goal

Build **production-ready tools** with proper error handling and structured outputs!

### What We'll Build:

✅ **Structured Output Tools**: Pydantic models for type safety  
✅ **Database Tools**: Query customer and order data  
✅ **External API Tools**: Payment and notification systems  
✅ **Chain-of-Thought**: Multi-step reasoning  
✅ **Error Handling**: Retries and fallbacks  

### Why This Matters:

Production agents need:
- **Type Safety**: Prevent errors with validated inputs/outputs
- **Reliability**: Handle failures gracefully
- **Integration**: Work with real databases and APIs
- **Observability**: Track what agents are doing

Let's make BakeryAI production-ready! 🚀

In [1]:
!pip install -q langchain langchain-openai langgraph
!pip install -q pandas python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 960.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 7.1 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Optional
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
!git clone https://github.com/IvanReznikov/mdx-langchain-conclave

Cloning into 'mdx-langchain-conclave'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 6), reused 24 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 240.64 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [4]:
from google.colab import userdata
import os

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY") or default_key


set_openai_api_key()
#set_openai_api_key("sk-...")

In [5]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [6]:
try:
    cakes_df = pd.read_csv('/content/mdx-langchain-conclave/data/cake_descriptions.csv', encoding='cp1252')
    customers_df = pd.read_csv('/content/mdx-langchain-conclave/data/customers.csv', encoding='cp1252')
    orders_df = pd.read_csv('/content/mdx-langchain-conclave/data/orders.csv', encoding='cp1252')
    print(f"✅ Loaded: {len(cakes_df)} products, {len(customers_df)} customers, {len(orders_df)} orders")
except FileNotFoundError:
    print("⚠️  Using sample data")
    cakes_df = pd.DataFrame({'Name': ['Chocolate Cake'], 'Available': [True]})
    customers_df = pd.DataFrame({'client_id': ['C001'], 'client_name': ['Alice'], 'loyalty_points': [100]})
    orders_df = pd.DataFrame({'order_id': ['O001'], 'status': ['delivered']})

✅ Loaded: 22 products, 50 customers, 100 orders


## 1. Structured Output Tools with Pydantic

Use Pydantic models for type-safe tool inputs and outputs.

In [7]:
# Define input/output schemas

class CustomerLookupInput(BaseModel):
    """Input for customer lookup tool"""
    query: str = Field(description="Customer name, email, or ID")

class CustomerInfo(BaseModel):
    """Customer information output"""
    client_id: str
    client_name: str
    client_type: str = "individual"
    email: Optional[str] = None
    loyalty_points: int = 0
    area: Optional[str] = None

class OrderInput(BaseModel):
    """Input for creating an order"""
    customer_id: str = Field(description="Customer ID")
    product_name: str = Field(description="Product name")
    quantity: int = Field(default=1, ge=1, le=10, description="Quantity (1-10)")
    delivery_date: str = Field(description="Delivery date")
    special_instructions: Optional[str] = Field(default=None, description="Special instructions")

    @validator('quantity')
    def validate_quantity(cls, v):
        if v < 1 or v > 10:
            raise ValueError('Quantity must be between 1 and 10')
        return v

class OrderConfirmation(BaseModel):
    """Order confirmation output"""
    order_id: str
    customer_name: str
    items: List[str]
    total_amount: float
    delivery_date: str
    status: str = "confirmed"

print("✅ Pydantic schemas defined")

✅ Pydantic schemas defined


## 2. Database Query Tools

Create tools that query customer and order databases.

In [8]:
@tool
def lookup_customer(query: str) -> str:
    """Look up customer information by name, email, or ID.

    Args:
        query: Customer identifier (name, email, or ID)

    Returns:
        Customer information as formatted string
    """
    # Search in customer database
    query_lower = query.lower()

    # Try matching by ID
    if 'client_id' in customers_df.columns:
        customer = customers_df[customers_df['client_id'].str.lower() == query_lower]
    else:
        customer = pd.DataFrame()

    # Try matching by name if ID search failed
    if customer.empty and 'client_name' in customers_df.columns:
        customer = customers_df[customers_df['client_name'].str.lower().str.contains(query_lower, na=False)]

    # Try matching by email
    if customer.empty and 'email' in customers_df.columns:
        customer = customers_df[customers_df['email'].str.lower().str.contains(query_lower, na=False)]

    if customer.empty:
        return f"❌ Customer '{query}' not found in database."

    # Get first match
    cust = customer.iloc[0]

    result = f"""✅ Customer Found:
    - ID: {cust.get('client_id', 'N/A')}
    - Name: {cust.get('client_name', 'N/A')}
    - Type: {cust.get('client_type', 'individual')}
    - Email: {cust.get('email', 'N/A')}
    - Loyalty Points: {cust.get('loyalty_points', 0)}
    - Area: {cust.get('area', 'N/A')}"""

    return result

@tool
def get_order_history(customer_id: str, limit: int = 5) -> str:
    """Get recent order history for a customer.

    Args:
        customer_id: Customer ID
        limit: Maximum number of orders to return (default 5)

    Returns:
        Order history summary
    """
    if 'client_id' not in orders_df.columns:
        return "Order history not available"

    customer_orders = orders_df[orders_df['client_id'] == customer_id].head(limit)

    if customer_orders.empty:
        return f"No order history found for customer {customer_id}."

    result = f"📦 Recent Orders for {customer_id}:\n\n"

    for idx, order in customer_orders.iterrows():
        result += f"Order #{order.get('order_id', 'N/A')}:\n"
        result += f"  - Date: {order.get('order_date', 'N/A')}\n"
        result += f"  - Items: {order.get('items', 'N/A')}\n"
        result += f"  - Total: ${order.get('total_amount', 0)}\n"
        result += f"  - Status: {order.get('status', 'N/A')}\n\n"

    return result

@tool
def check_order_status(order_id: str) -> str:
    """Check the current status of an order.

    Args:
        order_id: Order ID to check

    Returns:
        Order status and tracking information
    """
    if 'order_id' not in orders_df.columns:
        return "Order tracking not available"

    order = orders_df[orders_df['order_id'] == order_id]

    if order.empty:
        return f"❌ Order {order_id} not found."

    order_info = order.iloc[0]

    status_map = {
        'pending': '🟡 Pending - Order received, preparing',
        'preparing': '🟠 Preparing - Being baked fresh',
        'ready': '🟢 Ready - Ready for delivery',
        'out_for_delivery': '🚚 Out for Delivery - On the way',
        'delivered': '✅ Delivered - Completed',
        'cancelled': '❌ Cancelled'
    }

    status = order_info.get('status', 'unknown')
    status_display = status_map.get(status.lower(), status)

    result = f"""📦 Order Status for #{order_id}:

    Status: {status_display}
    Customer: {order_info.get('client_name', 'N/A')}
    Items: {order_info.get('items', 'N/A')}
    Total: ${order_info.get('total_amount', 0)}
    Delivery Date: {order_info.get('delivery_date', 'N/A')}
    """

    return result

# Test database tools
print("\n🔍 Testing Database Tools:\n")
if not customers_df.empty:
    first_customer = customers_df.iloc[0]['client_name'] if 'client_name' in customers_df.columns else customers_df.iloc[0]['client_id']
    print(lookup_customer.invoke({"query": first_customer}))


🔍 Testing Database Tools:

✅ Customer Found:
    - ID: C001
    - Name: Fatima Al Mazrouei
    - Type: Individual
    - Email: fatima.al mazrouei@email.com
    - Loyalty Points: 2
    - Area: JBR


## 3. External API Simulation Tools

Simulate payment processing and notification APIs.

In [9]:
@tool
def process_payment(customer_id: str, amount: float, payment_method: str = "card") -> str:
    """Process payment for an order.

    Args:
        customer_id: Customer ID
        amount: Payment amount
        payment_method: Payment method (card, cash, wallet)

    Returns:
        Payment confirmation
    """
    # Simulate payment processing
    import random
    import time

    time.sleep(0.5)  # Simulate API delay

    # 95% success rate
    success = random.random() < 0.95

    if success:
        transaction_id = f"TXN{random.randint(100000, 999999)}"
        return f"""✅ Payment Successful!

        Transaction ID: {transaction_id}
        Customer: {customer_id}
        Amount: ${amount:.2f}
        Method: {payment_method}
        Status: Approved
        """
    else:
        return f"""❌ Payment Failed

        Reason: Insufficient funds / Card declined
        Please try a different payment method.
        """

@tool
def send_confirmation_email(customer_email: str, order_id: str, order_details: str) -> str:
    """Send order confirmation email to customer.

    Args:
        customer_email: Customer's email address
        order_id: Order ID
        order_details: Order details to include

    Returns:
        Email send status
    """
    # Simulate email sending
    import time
    time.sleep(0.3)

    return f"""📧 Email Sent Successfully!

    To: {customer_email}
    Subject: Order Confirmation - #{order_id}

    Email Contents:
    {order_details}

    Status: Delivered to inbox
    """

@tool
def send_sms_notification(phone: str, message: str) -> str:
    """Send SMS notification to customer.

    Args:
        phone: Customer's phone number
        message: SMS message content

    Returns:
        SMS send status
    """
    import time
    time.sleep(0.2)

    return f"""📱 SMS Sent!

    To: {phone}
    Message: {message[:100]}...
    Status: Delivered
    """

# Test API tools
print("\n💳 Testing API Tools:\n")
print(process_payment.invoke({"customer_id": "C001", "amount": 75.50, "payment_method": "card"}))


💳 Testing API Tools:

✅ Payment Successful!
        
        Transaction ID: TXN253305
        Customer: C001
        Amount: $75.50
        Method: card
        Status: Approved
        


## 4. Chain-of-Thought Agent

Create an agent that shows its reasoning process.

In [10]:
# Collect all tools
all_tools = [
    lookup_customer,
    get_order_history,
    check_order_status,
    process_payment,
    send_confirmation_email,
    send_sms_notification
]

# Create prompt with chain-of-thought
system_prompt = """You are BakeryAI, an intelligent customer service agent.

When handling customer requests:
1. Think step-by-step about what information you need
2. Use tools to gather that information
3. Process the information logically
4. Provide clear, helpful responses

Always be friendly, professional, and thorough.
If you need to use multiple tools, explain your reasoning.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create agent
agent = create_openai_functions_agent(
    llm=llm,
    tools=all_tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=all_tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

print("✅ Chain-of-Thought Agent Ready!")

✅ Chain-of-Thought Agent Ready!


## 5. Complex Multi-Step Workflow Test

In [11]:
# Test: Complete order flow
print("🎯 Test: Complete Order Workflow\n")
print("=" * 70)

if not customers_df.empty:
    first_customer_name = customers_df.iloc[0]['client_name'] if 'client_name' in customers_df.columns else "Alice"

    result = agent_executor.invoke({
        "input": f"""Look up customer {first_customer_name}, show me their order history,
        and tell me their loyalty points."""
    })

    print("\n" + "="*70)
    print("📝 FINAL RESPONSE:")
    print("="*70)
    print(result['output'])
else:
    print("⚠️  No customer data available for testing")

🎯 Test: Complete Order Workflow



> Entering new AgentExecutor chain...

Invoking: `lookup_customer` with `{'query': 'Fatima Al Mazrouei'}`


✅ Customer Found:
    - ID: C001
    - Name: Fatima Al Mazrouei
    - Type: Individual
    - Email: fatima.al mazrouei@email.com
    - Loyalty Points: 2
    - Area: JBR
Invoking: `get_order_history` with `{'customer_id': 'C001'}`


📦 Recent Orders for C001:

Order #ORD0015:
  - Date: 2024-09-16
  - Items: Vanilla Bean Panna Cotta (4x25 AED)
  - Total: $100
  - Status: Pending

Here is the information for Fatima Al Mazrouei:

- **Loyalty Points:** 2
- **Recent Order History:**
  - **Order #ORD0015:**
    - **Date:** 2024-09-16
    - **Items:** Vanilla Bean Panna Cotta (4x25 AED)
    - **Total:** $100
    - **Status:** Pending

If you need further assistance, feel free to ask!

> Finished chain.

📝 FINAL RESPONSE:
Here is the information for Fatima Al Mazrouei:

- **Loyalty Points:** 2
- **Recent Order History:**
  - **Order #ORD0015:**
    - **Da

In [12]:
# Test: Order tracking
print("\n\n🎯 Test: Order Status Check\n")
print("=" * 70)

if not orders_df.empty:
    first_order_id = orders_df.iloc[0]['order_id'] if 'order_id' in orders_df.columns else "O001"

    result = agent_executor.invoke({
        "input": f"Check the status of order {first_order_id}"
    })

    print("\n" + "="*70)
    print("📝 FINAL RESPONSE:")
    print("="*70)
    print(result['output'])
else:
    print("⚠️  No order data available for testing")



🎯 Test: Order Status Check



> Entering new AgentExecutor chain...

Invoking: `check_order_status` with `{'order_id': 'ORD0037'}`


📦 Order Status for #ORD0037:
    
    Status: Completed
    Customer: Thomas Anderson
    Items: Torta della Nonna Amore (2x185 AED)
    Total: $395
    Delivery Date: 2024-11-01
    The status of order #ORD0037 is "Completed." Here are the details:

- **Customer:** Thomas Anderson
- **Items:** Torta della Nonna Amore (2x185 AED)
- **Total:** $395
- **Delivery Date:** 2024-11-01

If you need any further assistance, feel free to ask!

> Finished chain.

📝 FINAL RESPONSE:
The status of order #ORD0037 is "Completed." Here are the details:

- **Customer:** Thomas Anderson
- **Items:** Torta della Nonna Amore (2x185 AED)
- **Total:** $395
- **Delivery Date:** 2024-11-01

If you need any further assistance, feel free to ask!


## 6. Error Handling and Retries

In [13]:
@tool
def unreliable_api_call(data: str) -> str:
    """Simulate an unreliable API that sometimes fails.

    Args:
        data: Data to send to API

    Returns:
        API response or error
    """
    import random

    if random.random() < 0.3:  # 30% failure rate
        raise Exception("API timeout - please try again")

    return f"✅ API call successful: {data}"

# Tool with retry logic
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=1, max=4)
)
def call_external_api_with_retry(data: str) -> str:
    """Call external API with automatic retries"""
    return unreliable_api_call.invoke({"data": data})

# Test retry logic
print("Testing retry logic...")
try:
    result = call_external_api_with_retry("test data")
    print(result)
except Exception as e:
    print(f"Failed after retries: {e}")

Testing retry logic...
✅ API call successful: test data


## 7. Observability: Tracking Agent Actions

In [14]:
# Extract intermediate steps
def analyze_agent_execution(result):
    """Analyze what the agent did during execution"""

    print("\n📊 AGENT EXECUTION ANALYSIS")
    print("=" * 70)

    if 'intermediate_steps' not in result:
        print("No intermediate steps available")
        return

    steps = result['intermediate_steps']

    print(f"\nTotal Steps: {len(steps)}")
    print(f"\nStep-by-Step Breakdown:\n")

    for i, (action, observation) in enumerate(steps, 1):
        print(f"Step {i}:")
        print(f"  🔧 Tool: {action.tool}")
        print(f"  📥 Input: {action.tool_input}")
        print(f"  📤 Output: {str(observation)[:100]}...")
        print()

    # Calculate metrics
    tool_usage = {}
    for action, _ in steps:
        tool_usage[action.tool] = tool_usage.get(action.tool, 0) + 1

    print("Tool Usage Summary:")
    for tool, count in tool_usage.items():
        print(f"  - {tool}: {count} times")

# Run agent and analyze
if not customers_df.empty:
    first_customer = customers_df.iloc[0]['client_name'] if 'client_name' in customers_df.columns else "Customer"
    result = agent_executor.invoke({
        "input": f"Look up {first_customer} and tell me about their account"
    })

    analyze_agent_execution(result)



> Entering new AgentExecutor chain...

Invoking: `lookup_customer` with `{'query': 'Fatima Al Mazrouei'}`


✅ Customer Found:
    - ID: C001
    - Name: Fatima Al Mazrouei
    - Type: Individual
    - Email: fatima.al mazrouei@email.com
    - Loyalty Points: 2
    - Area: JBRHere is the account information for Fatima Al Mazrouei:

- **Customer ID:** C001
- **Name:** Fatima Al Mazrouei
- **Type:** Individual
- **Email:** fatima.almazrouei@email.com
- **Loyalty Points:** 2
- **Area:** JBR

If you need any further assistance or details, feel free to ask!

> Finished chain.

📊 AGENT EXECUTION ANALYSIS

Total Steps: 1

Step-by-Step Breakdown:

Step 1:
  🔧 Tool: lookup_customer
  📥 Input: {'query': 'Fatima Al Mazrouei'}
  📤 Output: ✅ Customer Found:
    - ID: C001
    - Name: Fatima Al Mazrouei
    - Type: Individual
    - Email: ...

Tool Usage Summary:
  - lookup_customer: 1 times


## 🎯 Exercise 3: Build an Order Placement Tool

**Task**: Create a comprehensive order placement tool that:
1. Validates customer exists
2. Checks product availability
3. Calculates total with delivery
4. Processes payment
5. Sends confirmation email
6. Returns structured order confirmation

In [15]:
@tool
def place_complete_order(
    customer_id: str,
    product_name: str,
    quantity: int,
    delivery_date: str,
    payment_method: str = "card"
) -> str:
    """Place a complete order with all validations and processing.

    Args:
        customer_id: Customer ID
        product_name: Product to order
        quantity: Quantity to order
        delivery_date: Desired delivery date
        payment_method: Payment method (card/cash/wallet)

    Returns:
        Complete order confirmation
    """
    # TODO: Implement complete order flow
    # Hint: Use the tools we created above in sequence
    # 1. Validate customer
    # 2. Check product availability
    # 3. Calculate total
    # 4. Process payment
    # 5. Generate order ID
    # 6. Send confirmation

    pass

# Test your tool
# place_complete_order.invoke(...)

## 🎯 Exercise 4: Build a Customer Service Escalation System

**Task**: Create tools and logic for handling complaints:
1. `detect_complaint_sentiment()` - Analyze if customer is upset
2. `log_complaint()` - Record complaint in system
3. `calculate_compensation()` - Determine discount/refund
4. `escalate_to_manager()` - Flag for human review

In [16]:
# TODO: Implement complaint handling system

@tool
def detect_complaint_sentiment(message: str) -> str:
    """Detect if customer message is a complaint and severity."""
    # TODO: Use LLM to analyze sentiment
    pass

# Add more complaint handling tools

## Summary: What We Built

### ✅ Session 2.2 Achievements:

1. **Structured Tools**: Pydantic models for type safety
2. **Database Integration**: Customer and order queries
3. **External APIs**: Payment and notification simulation
4. **Chain-of-Thought**: Multi-step reasoning
5. **Error Handling**: Retries and fallbacks
6. **Observability**: Tracking agent actions

### 🔧 BakeryAI Enhanced Tools:

✨ `lookup_customer` - Database queries  
✨ `get_order_history` - Historical data access  
✨ `check_order_status` - Real-time tracking  
✨ `process_payment` - Payment integration  
✨ `send_confirmation_email` - Notifications  
✨ `send_sms_notification` - Multi-channel alerts  

### 🚀 Next: Notebook 2.3

We'll introduce **LangGraph** for:
- Complex multi-agent workflows
- State management across steps
- Conditional routing
- Human-in-the-loop patterns